# Combustion Chemistry in Isobaric Reactor: Generatign Training Database in the Principal Component Space

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')


### Importing External Libraries

import numpy                             as np
import pandas                            as pd
pd.options.mode.chained_assignment = None  # default='warn'


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


# from scipy.integrate import solve_ivp
# import pyDOE    
# from PCAfold         import PCA          as PCAA

3.8.11 (default, Jun 29 2021, 03:17:05) 
[Clang 12.0.0 (clang-1200.0.32.29)]


Missing colon in file PosixPath('/Users/sventur/.matplotlib/stylelib/paper_2columns.mplstyle'), line 1 ('f#### MATPLOTLIBRC FORMAT')


## Defining Input Variables

In [2]:
OutputDir    = os.path.join(WORKSPACE_PATH, 'ROMNet/Data/0DReact_Isobaric_50Cases_H2/')


NVarsRed     = 10

tStratch     = 1.
SOLVER       = 'BDF'

valid_perc   = 20.

FixedMinVal  = 1.e-14

n_ics        = 500


ColorVec     = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

In [3]:
Vars     = []
for iVars in range(1,NVarsRed+1):
    Vars.append('PC_'+str(iVars))

Vars0    = []
for iVars in range(1,NVarsRed+1):
    Vars0.append('PC0_'+str(iVars))
    
SVars    = []
for iVars in range(1,NVarsRed+1):
    SVars.append('SPC_'+str(iVars))

InputVar = ['t'] + Vars0

# FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
# Data         = pd.read_csv(FileName, header=0)
# SimIdxs      = Data.to_numpy(int)


# FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
# Data         = pd.read_csv(FileName)
# ICVecs       = Data.to_numpy()
# n_ics         = len(ICVecs)


# P0Vec        = []
# EqRatio0Vec  = []
# T0Vec        = []
# for iC in range(n_ics):
#     Nts      = SimIdxs[iC,1]-SimIdxs[iC,0]
#     P0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,0])
#     EqRatio0Vec.append( np.ones(Nts, dtype=np.float64)*ICVecs[iC,1])
#     T0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,2])
# P0Vec        = np.concatenate(P0Vec)[...,np.newaxis]
# EqRatio0Vec  = np.concatenate(EqRatio0Vec)[...,np.newaxis]
# T0Vec        = np.concatenate(T0Vec)[...,np.newaxis]


# for iC in range(n_ics):
#     Nts              = SimIdxs[iC,1]-SimIdxs[iC,0]
#     FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
#     DataTemp         = pd.read_csv(FileName, header=0)
#     DataTemp         = DataTemp[DataTemp['t'] == 0.][Vars]
#     DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
#     if (iC==0):
#         Data0        = DataTemp
#     else:
#         Data0        = pd.concat([Data0, DataTemp], axis=0)
# Data0.columns = Vars0
# Data0.reset_index(drop=True, inplace=True)


for iC in range(n_ics):
    try:
        FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
        DataNew          = pd.read_csv(FileName, header=0)
        Nts              = len(DataNew)
        DataTemp         = DataNew.iloc[[0]][Vars] #DataTemp.iloc[1][Vars]
        DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
        DataTemp.reset_index(drop=True, inplace=True)
        DataNew[Vars0]   = DataTemp
        DataTemp['t']    = DataNew['t']
        DataNew['t']     = np.maximum(DataNew['t'].to_numpy() - DataTemp['t'].iloc[[0]].to_numpy(), FixedMinVal)
        if (iC==0):
            Data         = DataNew
        else:
            Data         = pd.concat([Data, DataNew], axis=0)
    except:
        pass
Data.reset_index(drop=True, inplace=True)

In [4]:
Data

,t,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,...,PC0_1,PC0_2,PC0_3,PC0_4,PC0_5,PC0_6,PC0_7,PC0_8,PC0_9,PC0_10
0,1.000000e-14,-11.681166,0.097983,0.532686,0.047136,0.032925,0.000558,-0.010958,0.004158,0.000883,...,-11.681166,0.097983,0.532686,0.047136,0.032925,0.000558,-0.010958,0.004158,0.000883,0.000498
1,1.384931e-07,-11.681830,0.097973,0.532668,0.047160,0.032929,0.000549,-0.011009,0.002583,0.000856,...,-11.681166,0.097983,0.532686,0.047136,0.032925,0.000558,-0.010958,0.004158,0.000883,0.000498
2,1.924229e-07,-11.682090,0.097969,0.532660,0.047169,0.032931,0.000546,-0.011028,0.001979,0.000846,...,-11.681166,0.097983,0.532686,0.047136,0.032925,0.000558,-0.010958,0.004158,0.000883,0.000498
3,2.673532e-07,-11.682450,0.097962,0.532648,0.047183,0.032933,0.000542,-0.011055,0.001152,0.000832,...,-11.681166,0.097983,0.532686,0.047136,0.032925,0.000558,-0.010958,0.004158,0.000883,0.000498
4,3.714616e-07,-11.682948,0.097951,0.532628,0.047202,0.032937,0.000536,-0.011092,0.000028,0.000814,...,-11.681166,0.097983,0.532686,0.047136,0.032925,0.000558,-0.010958,0.004158,0.000883,0.000498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2.674764e-01,14.960013,-0.297099,0.153008,-0.012274,-0.006435,-0.023437,-0.009452,0.000509,-0.000505,...,-24.698149,-0.047986,0.386769,0.070095,-0.000460,-0.010752,0.006997,0.003276,0.000569,0.000317
2496,3.716328e-01,14.960013,-0.297099,0.153008,-0.012274,-0.006435,-0.023437,-0.009452,0.000509,-0.000505,...,-24.698149,-0.047986,0.386769,0.070095,-0.000460,-0.010752,0.006997,0.003276,0.000569,0.000317
2497,5.163481e-01,14.960013,-0.297099,0.153008,-0.012274,-0.006435,-0.023437,-0.009452,0.000509,-0.000505,...,-24.698149,-0.047986,0.386769,0.070095,-0.000460,-0.010752,0.006997,0.003276,0.000569,0.000317
2498,7.174162e-01,14.960013,-0.297099,0.153008,-0.012274,-0.006435,-0.023437,-0.009452,0.000509,-0.000505,...,-24.698149,-0.047986,0.386769,0.070095,-0.000460,-0.010752,0.006997,0.003276,0.000569,0.000317


In [5]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/')
except:
    pass

In [6]:
data_id    = 'pts'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[['t'] + Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [7]:
data_id    = 'res'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput        = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = DataNoZero[['t'] + Vars]
DataOutput[Vars] = DataOutput[Vars].to_numpy() * 0.
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [8]:
data_id = 'ics'

Data_ics                     = Data[Data['t'] == FixedMinVal]
n_points_ics                 = len(Data_ics)

idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=41)

n_valid                      = len(valid_idx)
n_train                      = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput      = Data_ics[InputVar]
DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput     = Data_ics[['t'] + Vars]
DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [9]:
# data_id = 'scs'

# Mask                         = [i for i in range(1,len(Data)-1) if Data.iloc[i-51]['t'] == FixedMinVal]
# Data_ics                     = Data.iloc[Mask]
# n_points_ics                 = len(Data_ics)
# idx_ics                      = np.arange(n_points_ics)
# train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=41)

# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
# except:
#     pass
# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
# except:
#     pass

# DataInput      = Data_ics[InputVar]
# DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
# DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

# DataOutput     = Data_ics[['t'] + Vars]
# DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
# DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [10]:
# data_id = 'fcs'

# Mask                         = [i for i in range(-1,len(Data)-1) if Data.iloc[i+1]['t'] == FixedMinVal]
# Data_ics                     = Data.iloc[Mask]

# n_points_ics                 = len(Data_ics)
# idx_ics                      = np.arange(n_points_ics)
# train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=43)

# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
# except:
#     pass
# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
# except:
#     pass

# DataInput      = Data_ics[InputVar]
# DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
# DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

# DataOutput       = Data_ics[['t'] + Vars]
# DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
# DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)